<a href="https://colab.research.google.com/github/Junior11995/Challenge-TelecomX-Parte2-Prediccion/blob/main/Challenge_TelecomX_Parte2_Prediccion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# introducción

# 📊 Challenge Telecom X - Parte 2: Predicción de Evasión de Clientes

## 🎯 Introducción
Este desafío forma parte de la segunda etapa del proyecto Telecom X.  
En la Parte 1 trabajamos como analistas de datos explorando, limpiando y generando insights sobre la evasión de clientes (*churn*).  
Ahora, en la Parte 2, nuestro rol es de **científicos de datos** y el objetivo principal es **construir modelos predictivos** que permitan anticipar qué clientes tienen mayor riesgo de cancelar los servicios.

## 📌 Objetivos
- Preparar y separar adecuadamente los datos para modelado (train/test).
- Analizar correlaciones y seleccionar variables relevantes.
- Aplicar conceptos de estadística y regresión lineal.
- Entrenar y evaluar modelos de **Machine Learning de clasificación** para predecir el churn.
- Identificar las variables que más influyen en la evasión.
- Generar un informe con conclusiones y recomendaciones estratégicas para la empresa.

## 🚀 Importancia
Este desafío conecta la teoría con la práctica en un caso real de negocio.  
Al final, entregaremos un **notebook claro y documentado** junto con un **repositorio en GitHub** que refleje cómo la ciencia de datos puede aplicarse para resolver problemas empresariales críticos.


#  Preparación de los Datos

En esta etapa transformamos el dataset tratado en la Parte 1 en un conjunto listo para Machine Learning:

- Eliminamos columnas irrelevantes (`customerid`).
- Convertimos variables categóricas a numéricas (OneHotEncoding).
- Separamos variables explicativas (`X`) de la variable objetivo (`y = churn`).
- Dividimos en datos de entrenamiento y prueba.
- Estandarizamos variables numéricas.


In [4]:
# ============================================
# CARGA ROBUSTA DEL DATASET TRATADO (PARTE 1)
# ============================================
import pandas as pd

LOCAL_PATHS = ["telecom_clean.csv", "/content/telecom_clean.csv"]

df_ml = None
for p in LOCAL_PATHS:
    try:
        df_ml = pd.read_csv(p)
        print(f"✅ Cargado desde: {p}")
        break
    except Exception:
        pass

if df_ml is None:
    try:
        df_ml = pd.read_csv(GITHUB_RAW_URL)
        print(f"✅ Cargado desde URL: {GITHUB_RAW_URL}")
    except Exception as e:
        raise FileNotFoundError(
            "No se pudo cargar 'telecom_clean.csv'. "
            "Asegúrate de tenerlo en el directorio actual o ajusta GITHUB_RAW_URL."
        )

print("Shape:", df_ml.shape)
display(df_ml.head())


✅ Cargado desde: telecom_clean.csv
Shape: (7267, 23)


,id_cliente,evadio,customer_gender,customer_seniorcitizen,customer_partner,customer_dependents,customer_tenure,phone_phoneservice,phone_multiplelines,internet_internetservice,...,internet_techsupport,internet_streamingtv,internet_streamingmovies,account_contract,account_paperlessbilling,account_paymentmethod,account_charges.monthly,account_charges.total,_monthlycharges_std,gasto_diario
0,0002-orfbo,0.0,NaN,0,yes,NaN,9,1.0,0.0,dsl,...,1.0,1.0,0.0,one year,1.0,mailed check,65.6,593.30,NaN,NaN
1,0003-mknfe,0.0,NaN,0,no,NaN,9,1.0,1.0,dsl,...,0.0,0.0,1.0,month-to-month,0.0,mailed check,59.9,542.40,NaN,NaN
2,0004-tlhlj,1.0,NaN,0,no,NaN,4,1.0,0.0,fiber optic,...,0.0,0.0,0.0,month-to-month,1.0,electronic check,73.9,280.85,NaN,NaN
3,0011-igkff,1.0,NaN,1,yes,NaN,13,1.0,0.0,fiber optic,...,0.0,1.0,1.0,month-to-month,1.0,electronic check,98.0,1237.85,NaN,NaN
4,0013-exchz,1.0,NaN,1,yes,NaN,3,1.0,0.0,fiber optic,...,1.0,1.0,0.0,month-to-month,1.0,mailed check,83.9,267.40,NaN,NaN


# PREPARACIÓN DE LOS DATOS PARA ML

In [5]:
# ============================================
# 🛠️ PREPARACIÓN DE LOS DATOS PARA ML
#  - Elimina columnas irrelevantes
#  - Detecta/normaliza la variable objetivo
#  - Split train/test con estratificación
#  - Preprocesador (OneHot + StandardScaler)
# ============================================
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

df_prep = df_ml.copy()

# 1) Drop de columnas irrelevantes (ajusta la lista si tienes más metas/IDs)
drop_cols = [c for c in df_prep.columns if c.lower() in {"customerid", "id_cliente"}]
if drop_cols:
    df_prep = df_prep.drop(columns=drop_cols)

# 2) Detectar columna target (acepta 'churn' o 'evadio')
lower_cols = {c.lower(): c for c in df_prep.columns}
if "churn" in lower_cols:
    target_col = lower_cols["churn"]
elif "evadio" in lower_cols:
    target_col = lower_cols["evadio"]
else:
    raise KeyError("No se encontró la columna objetivo ('churn' o 'evadio').")

# 3) Normalizar target a 0/1
y_raw = df_prep[target_col]
y = (
    y_raw.astype(str).str.strip().str.lower()
      .map({"yes":1,"si":1,"sí":1,"true":1,"1":1, "no":0,"false":0,"0":0})
)
# Si ya viene numérica, convertir respetando valores
if y.isna().mean() > 0.5:
    y = pd.to_numeric(y_raw, errors="coerce")
y = y.astype(float)

# 4) Definir X (el resto de columnas)
X = df_prep.drop(columns=[target_col])

# 5) Columnas categóricas y numéricas
cat_cols = X.select_dtypes(include=["object", "category"]).columns.tolist()
num_cols = X.select_dtypes(include=[np.number]).columns.tolist()

# 6) Balance de clases (chequeo rápido)
print("Distribución de la clase (target):")
print(y.value_counts(dropna=False))
print("\nDistribución %:")
print((y.value_counts(normalize=True)*100).round(2))

# 7) Split train/test estratificado
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

print("\nShapes → X_train:", X_train.shape, " X_test:", X_test.shape)

# 8) Preprocesador: escala numéricas y OneHot en categóricas
preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), num_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), cat_cols),
    ],
    remainder="drop",
    verbose_feature_names_out=False,
)

print("\n✅ Preprocesador listo (StandardScaler + OneHotEncoder).")
print("Numéricas:", len(num_cols), "| Categóricas:", len(cat_cols))

Distribución de la clase (target):
evadio
0.0    5398
1.0    1869
Name: count, dtype: int64

Distribución %:
evadio
0.0    74.28
1.0    25.72
Name: proportion, dtype: float64

Shapes → X_train: (5813, 21)  X_test: (1454, 21)

✅ Preprocesador listo (StandardScaler + OneHotEncoder).
Numéricas: 17 | Categóricas: 4
